In [ ]:
# title: MapBiomas Soil
# subtitle: Prepare environmental covariates
# author: Alessandro Samuel-Rosa and Taciara Zborowski Horst
# data: 2025

# Import necessary libraries
import os
import pandas as pd

# Identify working directory, saving the path to a variable
src_dir = os.getcwd()
work_dir = os.path.dirname(src_dir)

# Read the TXT file 'data/12_soildata.txt' with the soil data from the 'data' folder processed
# in the previous script '12_prepare_soil_covars.R'
# Field separator: tab
file_path = os.path.join(work_dir, 'data', '12_soildata.txt')
soildata_df = pd.read_csv(file_path, sep='\t', low_memory=False)
print(soildata_df.shape)
# (54105, 70)

In [ ]:
# Read the TXT file 'data/13_soildata.txt' with the coordinates from the 'data' folder
# Field separator: tab. Decimal separator: comma
# file_path = os.path.join(work_dir, 'data', '13_soildata.txt')
# df2 = pd.read_csv(file_path, sep='\t', low_memory=False)
# print(df2.shape)

# Compare the two data frames to check if there were changes in the data
# Use equals() method to compare the two data frames
# The result is a boolean value, True if the data frames are equal, False otherwise
# We consider only the following variables in the comparison:
# dataset_id, observacao_id, coord_x, coord_y, data_ano
# df1 = soildata_df[['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']]
# df2 = df2[['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']]
# if df1.equals(df2):
#     print("The DataFrames are identical.")
# else:
#     print("The DataFrames are different.")
# del df1, df2
# 29683 layers (GET FROM PREVIOUS SCRIPT)

In [ ]:
# UNIQUE EVENTS WITH GEOGRAPHIC COORDINATES
target = 16566 # GET VALUE FROM PREVIOUS SCRIPT!

# Filter out soil layers without geographic coordinates
# coord_x and coord_y are the columns with the geographic coordinates
soildata_xy = soildata_df[soildata_df['coord_x'].notnull()]
soildata_xy = soildata_xy[soildata_xy['coord_y'].notnull()]

# Remove all duplicates based on the following columns:
# "dataset_id", "observacao_id", "coord_x", "coord_y", "data_ano"
# The first occurrence is kept, and the others are removed
soildata_xy = soildata_xy[['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']]
soildata_xy = soildata_xy.drop_duplicates()
print(
  'There should be', target, 'events:', target == soildata_xy.shape[0],
  '\nThere are', soildata_xy.shape[0], 'events')

In [ ]:
# Print the first and last 5 rows of the data frame
print(soildata_xy)

In [ ]:
# Import necessary libraries
import ee
import geemap
import numpy as np  # Import numpy to access np.nan

# Initialize the Earth Engine API
# ee.Authenticate()
ee.Initialize(project='mapbiomas-solos-workspace')

# 1. Clean the DataFrame: Replace all NaN values with None
soildata_xy_cleaned = soildata_xy.replace({np.nan: None})

# 2. Convert the *cleaned* DataFrame to an Earth Engine Feature Collection
soildata_fc = geemap.df_to_ee(soildata_xy_cleaned, latitude='coord_y', longitude='coord_x')
print(soildata_fc.size().getInfo(), 'features')

# 3. Function to split a feature collection (sampling points) into chunks
def split_sampling_points(fc, chunk_size):
    features = fc.toList(fc.size())
    chunks = [features.slice(i, i + chunk_size) for i in range(0, features.size().getInfo(), chunk_size)]
    return [ee.FeatureCollection(chunk) for chunk in chunks]
# Split the sample points into subsets of 1000 points each
# This is necessary to avoid timeout errors
chunk_size = 1000
soildata_chunks = split_sampling_points(soildata_fc, chunk_size)
print(len(soildata_chunks), 'chunks')

In [ ]:
# SoilGrids 250m (Hengl et al., 2017)
# WRB Soil Reference Groups at 250-m spatial resolution
wrb_path = "projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/WRB_ALL_SOILS_SOILGRIDS_30M_GAPFILL"
wrb_bands = ['Ferralsols', 'Histosols', 'Nitisols', 'Vertisols', 'Plinthosols', 'Arenosols',
             'Podzols', 'Chernozems', 'Phaeozems', 'Umbrisols', 'Leptosols', 'Regosols',
             'Gleysols', 'Planosols', 'Stagnosols', 'Alisols', 'Luvisols', 'Acrisols', 'Lixisols']
wrb_image = ee.Image(wrb_path).select(wrb_bands)
print(wrb_image.getInfo())

In [ ]:
# Black Soils (FAO, 2022)
# Probability of occurrence of black soils at 1-km spatial resolution
black_path = "projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/FAO_2022_BLACKSOIL_1KM"
black_image = ee.Image(black_path).rename('black_soils')
print(black_image.getInfo())

In [ ]:
# SoilGrids 250m (Poggio et al., 2021)
# Soil properties at 250-m spatial resolution at six standard depths (0-5cm, 5-15cm, 15-30cm,
# 30-60cm, 60-100cm, 100-200cm)

# Soil bulk density (bdod_mean)
bdod_path = "projects/soilgrids-isric/bdod_mean"
bdod_bands = ['bdod_0-5cm_mean', 'bdod_5-15cm_mean', 'bdod_15-30cm_mean',
              'bdod_30-60cm_mean', 'bdod_60-100cm_mean', 'bdod_100-200cm_mean']
bdod_image = ee.Image(bdod_path).select(bdod_bands)

# Clay content (clay_mean)
clay_path = "projects/soilgrids-isric/clay_mean"
clay_bands = ['clay_0-5cm_mean', 'clay_5-15cm_mean', 'clay_15-30cm_mean',
              'clay_30-60cm_mean', 'clay_60-100cm_mean', 'clay_100-200cm_mean']
clay_image = ee.Image(clay_path).select(clay_bands)

# Sand content (sand_mean)
sand_path = "projects/soilgrids-isric/sand_mean"
sand_bands = ['sand_0-5cm_mean', 'sand_5-15cm_mean', 'sand_15-30cm_mean', 
              'sand_30-60cm_mean', 'sand_60-100cm_mean', 'sand_100-200cm_mean']
sand_image = ee.Image(sand_path).select(sand_bands)

# Soil organic carbon (soc_mean)
soc_path = "projects/soilgrids-isric/soc_mean"
soc_bands = ['soc_0-5cm_mean', 'soc_5-15cm_mean', 'soc_15-30cm_mean',
             'soc_30-60cm_mean', 'soc_60-100cm_mean', 'soc_100-200cm_mean']
soc_image = ee.Image(soc_path).select(soc_bands)

# Volume of coarse fragments (cfvo_mean)
cfvo_path = "projects/soilgrids-isric/cfvo_mean"
cfvo_bands = ['cfvo_0-5cm_mean', 'cfvo_5-15cm_mean', 'cfvo_15-30cm_mean',
              'cfvo_30-60cm_mean', 'cfvo_60-100cm_mean', 'cfvo_100-200cm_mean']
cfvo_image = ee.Image(cfvo_path).select(cfvo_bands)

# Stack all soil property images into a single image
soil_image = (bdod_image
              .addBands(clay_image)
              .addBands(sand_image)
              .addBands(soc_image)
              .addBands(cfvo_image))
# Improve band names
soil_image = soil_image.select(
  soil_image.bandNames(),
  soil_image.bandNames().map(lambda b: ee.String(b).replace('_mean', '').replace('_0', '_00').replace('-', '_').replace('_5', '_05')))
# Print soil image information
print(soil_image.getInfo())

In [ ]:
# Altitude (Yamazaki et al., 2017)
# Digital Elevation Model (DEM)
dem_path = "MERIT/DEM/v1_0_3"
dem_image = ee.Image(dem_path).select('dem').round().rename('altitude')

# Height above the nearest drainage
hnd_path = "MERIT/Hydro/v1_0_1"
hnd_image = ee.Image(hnd_path).select('hnd').round().rename('hand')

# Stack MERIT images
merit_image = dem_image.addBands(hnd_image)
print(merit_image.getInfo())

In [ ]:
# Land surface variables (Amatulli et al., 2020, 2018)
# Geomorpho90m
geomorpho_path = "projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/OT_GEOMORPHOMETRY_90m"

# Slope
slope_image = ee.Image(geomorpho_path).select('slope').multiply(3.141593).divide(180).tan().multiply(100).round()

# Convergence
convergence_image = ee.Image(geomorpho_path).select('convergence').round()

# Compound Topographic Index (CTI)
# cti * 10
cti_image = ee.Image(geomorpho_path).select('cti').multiply(10).round()

# Eastness
# eastness * 100
eastness_image = ee.Image(geomorpho_path).select('eastness').multiply(100).round()

# Northness
# northness * 100
northness_image = ee.Image(geomorpho_path).select('northness').multiply(100).round()

# Profile Curvature
# pcurv * 1000
pcurv_image = ee.Image(geomorpho_path).select('pcurv').multiply(1000).round()

# Roughness
roughness_image = ee.Image(geomorpho_path).select('roughness').round()

# Stream Power Index (SPI)
spi_image = ee.Image(geomorpho_path).select('spi').add(1).log10().multiply(100).round()

# Elevation Standard Deviation
elev_stdev_image = ee.Image(geomorpho_path).select('elev_stdev').round()

# Maximum Multiscale Deviation
dev_magnitude_image = ee.Image(geomorpho_path).select('dev_magnitude').round()

# Geomorphon (geomorphological forms)
# geom
geom_image = ee.Image(geomorpho_path).select('geom').round().rename('geomorphon')

# Stack all land surface variable images into a single image
geomorpho_image = (slope_image
             .addBands(convergence_image)
             .addBands(cti_image)
             .addBands(eastness_image)
             .addBands(northness_image)
             .addBands(pcurv_image)
             .addBands(roughness_image)
             .addBands(spi_image)
             .addBands(elev_stdev_image)
             .addBands(dev_magnitude_image)
             .addBands(geom_image))
print(geomorpho_image.getInfo())

In [ ]:
# Structural Provinces (IBGE, 2019c)

# First level: Provinces
province_path = "projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/IBGE_PROVINCIAIS_ESTRUTURAIS_250mil_2025/IBGE_PROVINCIAS_250MIL_DUMMY"
province_image = ee.Image(province_path)
# Replace "_Provincia" with "_prov"
province_image = province_image.select(
  province_image.bandNames(),
  province_image.bandNames().map(lambda b: ee.String(b).replace('_Provincia', 'Prov', 'i').replace('_', '')))
print(province_image.getInfo())

# Second level: Subprovinces
subprovince_path = "projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/IBGE_PROVINCIAIS_ESTRUTURAIS_250mil_2025/subprovincias"
subprovince_image = ee.Image(subprovince_path)
# Replace "_Subprov" with "_subprov"
subprovince_image = subprovince_image.select(
  subprovince_image.bandNames(),
  subprovince_image.bandNames().map(lambda b: ee.String(b).replace('_Subprovincia', 'Subprov', 'i').replace('_', '')))
print(subprovince_image.getInfo())

# Stack all structural province images into a single image
geo_image = (province_image
             .addBands(subprovince_image))
print(geo_image.getInfo())

In [ ]:
# Distance to specific landforms (MapBiomas, 2024)

# Rock outcrops
rock_path = 'projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/DISTANCE_C10_v1/distance_afloramento_rochoso_c10_v1'
rock_image = ee.Image(rock_path).rename('dist2rock')

# Sand deposits (beaches, dunes, sandy areas)
sand_path = 'projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/DISTANCE_C10_v1/distance_praia_duna_areal_c10_v1'
sand_image = ee.Image(sand_path).rename('dist2sand')

# Stack all distance to landform images into a single image
dist_image = rock_image.addBands(sand_image)
print(dist_image.getInfo())

In [ ]:
# Extract covariate values to sampling points
# This takes about 15 minutes to run...

# Stack the images into a single multiband image
static_image = (wrb_image # Soil Reference Groups
                .addBands([
                    black_image, # Black Soils (FAO, 2022)
                    soil_image, # SoilGrids
                    merit_image, # MERIT DEM and HND
                    geomorpho_image, # Geomorphological Variables
                    geo_image, # Structural Provinces
                    dist_image # Distance to specific landforms
                ]))
                   
# Initialize an empty list to store DataFrames
dataframes = []

# Loop over each subset and sample the data
for chunk in soildata_chunks:
    sampled_points = geemap.extract_values_to_points(chunk, static_image, scale=250)
    sampled_df = geemap.ee_to_df(sampled_points)
    dataframes.append(sampled_df)

# Concatenate all DataFrames into a single DataFrame
static_df = pd.concat(dataframes, ignore_index=True)

# Print column names
print(static_df.columns)

# Check if number of rows of static_df is the same as that of soildata_xy
# If the number of rows is the same, the merge was successful
target = soildata_xy.shape[0]
print(
  '\nThere should be', target, 'events:',
  target == static_df.shape[0], '\nThere are', static_df.shape[0], 'events')

In [ ]:
# Compute summary statistics of the static covariates
summary = static_df.describe()
print(summary)

In [ ]:
# MapBiomas Land Use and Land Cover (LULC) Collection
# (this takes about 10 minutes to run)

from datetime import datetime

# MapBiomas LULC Collection
col9 = "collection9/mapbiomas_collection90_integration_v1"
col10 = "collection10/mapbiomas_brazil_collection10_integration_v2"

# Import the MapBiomas LULC
mapbiomas_path = "projects/mapbiomas-public/assets/brazil/lulc/" + col10
mapbiomas_image = ee.Image(mapbiomas_path)
# Remove classification_ prefix from band names
mapbiomas_image = mapbiomas_image.select(
  mapbiomas_image.bandNames(),
  mapbiomas_image.bandNames().map(lambda b: ee.String(b).replace('classification_', '')))
print('MapBiomas Image Info:')
print(mapbiomas_image.getInfo())

# Initialize an empty list to store DataFrames
dataframes = []

# Loop over each subset and sample the data
for chunk in soildata_chunks:
    sampled_points = geemap.extract_values_to_points(chunk, mapbiomas_image, scale=30)
    sampled_df = geemap.ee_to_df(sampled_points)
    dataframes.append(sampled_df)

In [ ]:
# Concatenate all DataFrames into a single DataFrame
mapbiomas_df = pd.concat(dataframes, ignore_index=True)

# Rename columns
# Remove classification_ prefix from column names
# mapbiomas_df.columns = mapbiomas_df.columns.str.replace('classification_', '')

# Get LULC class at the year of sampling (data_ano)
# Each columns of 'mapbiomas_df' stores the LULC in a given year. The column name is the year. The
# value in the column is the class code (int). Each row is a soil sampling point. The column
# 'data_ano' stores the year of soil sampling (float). We need to get the exact LULC class code for
# each sampling point based on the year of sampling.
def get_lulc_class(row):
    year_val = row['data_ano']
    if pd.isna(year_val):
        return pd.NA
    year = int(year_val)
    if year < 1985:
        return pd.NA
    year_str = str(year)
    return str(row[year_str]) if year_str in row.index else pd.NA
mapbiomas_df['lulc'] = mapbiomas_df.apply(get_lulc_class, axis=1)

# Drop columns with years as column names
current_year = datetime.now().year
years_to_drop = [str(year) for year in range(1985, current_year + 1)]
mapbiomas_df.drop(columns=years_to_drop, inplace=True, errors='ignore')

# Reclassify the land cover classes
# https://brasil.mapbiomas.org/codigos-de-legenda/
forest_codes = ['1.0', '3.0', '4.0', '5.0', '6.0', '49.0']
nonforest_codes = ['10.0', '11.0', '12.0', '32.0', '29.0', '50.0']
pasture_codes = ['15.0']
agrishort_codes = ['19.0', '39.0', '20.0', '40.0', '62.0', '41.0']
agrilong_codes = ['36.0', '46.0', '47.0', '35.0', '48.0']
forestry_codes = ['9.0']
mosaic_codes = ['21.0']
nonvegetation_codes = ['22.0', '23.0', '24.0', '30.0', '25.0', '26.0', '33.0', '31.0', '27.0']
na_codes = ['0', '0.0', 'nan']
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(forest_codes, 'forest')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(nonforest_codes, 'nonforest')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(pasture_codes, 'pasture')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(agrishort_codes, 'shortAgriculture')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(agrilong_codes, 'longAgriculture')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(forestry_codes, 'forestry')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(mosaic_codes, 'mosaic')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(nonvegetation_codes, 'nonvegetation')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(na_codes, pd.NA)

# Print the first 5 rows of the data frame
print(
    '\nFirst 5 rows of the MapBiomas LULC DataFrame with the lulc column:')
print(mapbiomas_df.head())

# Print summary of the land cover classes, including NA
print('\nDistribution of land cover/land use classes (including NA):')
print(mapbiomas_df['lulc'].value_counts(dropna=False))

# Check if number of rows of mapbiomas_df is the same as that of soildata_xy
# If the number of rows is the same, the sampling was successful
target = soildata_xy.shape[0]
print(
  '\nThere should be', target, 'events:',
  target == mapbiomas_df.shape[0], '\nThere are', mapbiomas_df.shape[0], 'events')

In [ ]:
# Fill empty cells in the 'lulc' column with 'unknown'
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].fillna('unknown')

# Merge data sampled from Earth Engine with the original soil data
# Keep all rows from soildata_df
merge_columns = ['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']
output_df = soildata_df.merge(static_df, on = merge_columns, how = 'left')
output_df = output_df.merge(mapbiomas_df, on = merge_columns, how = 'left')

# Check if number of rows of output_df is the same as that of soildata_df
# If the number of rows is the same, the merge was successful
target = soildata_df.shape[0]
print(
  '\nThere should be', target, 'layers:',
  target == output_df.shape[0], '\nThere are', output_df.shape[0], 'layers')

In [ ]:
# Write the output_df to a TXT file
# Field separator: tab. Decimal separator: comma
file_path = os.path.join(work_dir, 'data', '13_soildata.txt')
output_df.to_csv(file_path, sep='\t', index=False)